In [ ]:
# Необходимые импорты
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import datetime
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import (RandomizedSearchCV, cross_val_score, 
                                   StratifiedKFold, KFold)
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from scipy import stats
import joblib
import warnings
import os
import chardet
from charset_normalizer import from_bytes

# Настройки
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")

SEED = 42
np.random.seed(SEED)
random.seed(SEED)


# 1. Безопасная загрузка данных
def safe_load_data():
    try:
        train_path = "ads.csv"
        test_path = "test.tsv.csv"
        
        if not os.path.exists(train_path):
            raise FileNotFoundError(f"Файл {train_path} не найден")
        
        print("ЗАГРУЗКА ОБУЧАЮЩИХ ДАННЫХ...")
        train = load_with_best_encoding(train_path)
            
        if 'successful' not in train.columns:
            raise ValueError("В данных отсутствует целевая переменная 'successful'")
            
        if os.path.exists(test_path):
            print("\nЗАГРУЗКА ТЕСТОВЫХ ДАННЫХ...")
            test = load_with_best_encoding(test_path)
        else:
            print("Тестовые данные не найдены, будет использована пустая DataFrame")
            test = pd.DataFrame()
            
        return train, test
    
    except Exception as e:
        print(f"Ошибка при загрузке данных: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()

# 2. Предобработка данных
def preprocess_data(train, test):
    try:
        train = train.drop_duplicates().dropna(how='all')
        if not test.empty:
            test = test.drop_duplicates().dropna(how='all')
        
        for df in [train, test]:
            if not df.empty:
                for col in df.select_dtypes(include='object').columns:
                    try:
                        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',','.'), errors='raise')
                    except:
                        try:
                            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',',''), errors='raise')
                        except:
                            continue
        
        return train.drop(columns=['successful']), train['successful'], test
    
    except Exception as e:
        print(f"Ошибка при предобработке данных: {str(e)}")
        return pd.DataFrame(), pd.Series(), pd.DataFrame()

# 3. Анализ данных
def analyze_data(X, y):
    print("\n=== АНАЛИЗ ДАННЫХ ===")
    print(f"Размер обучающей выборки: {X.shape}")
    print("\nРаспределение классов:")
    print(y.value_counts(normalize=True))
    
    numeric_cols = X.select_dtypes(include=np.number).columns
    if len(numeric_cols) > 0:
        corr_matrix = pd.concat([X[numeric_cols], y], axis=1).corr()
        print("\nКорреляции с целевой переменной:")
        print(corr_matrix['successful'].sort_values(ascending=False))
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
        plt.title("Матрица корреляций")
        plt.tight_layout()
        plt.show()
    else:
        print("\nНет числовых признаков для анализа корреляций")

# 4. Создание упрощенного пайплайна
def create_preprocessor():
    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', RobustScaler())
    ])

    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    return ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, make_column_selector(dtype_include=np.number)),
            ('cat', categorical_transformer, make_column_selector(dtype_include=object))
        ],
        remainder='passthrough'
    )

# 5. Настройка моделей
def get_models():
    return {
        'LogisticRegression': {
            'model': LogisticRegression(max_iter=1000, random_state=SEED),
            'params': {
                'model__C': [0.1, 1.0, 10.0],
                'model__penalty': ['l2'],
                'model__solver': ['liblinear'],
                'model__class_weight': [None, 'balanced']
            }
        },
        'RandomForest': {
            'model': RandomForestClassifier(random_state=SEED),
            'params': {
                'model__n_estimators': [100, 200],
                'model__max_depth': [None, 10],
                'model__min_samples_split': [2, 5]
            }
        },
        'XGBoost': {
            'model': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=SEED),
            'params': {
                'model__n_estimators': [100, 200],
                'model__max_depth': [3, 5],
                'model__learning_rate': [0.01, 0.1]
            }
        },
        'LightGBM': {
            'model': LGBMClassifier(random_state=SEED, verbose=-1),
            'params': {
                'model__num_leaves': [31, 63],
                'model__learning_rate': [0.05, 0.1],
                'model__n_estimators': [100, 200]
            }
        }
    }

# 6. Обучение и оценка моделей
def train_and_evaluate(X, y, preprocessor, models):
    results = []
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
    
    # Бейзлайн модель
    baseline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(max_iter=1000, random_state=SEED))
    ])
    
    print("\n=== БЕЙЗЛАЙН МОДЕЛЬ ===")
    try:
        baseline_scores = cross_val_score(baseline, X, y, cv=cv, scoring='accuracy', n_jobs=1)
        baseline_accuracy = np.mean(baseline_scores)
        print(f"Baseline Accuracy: {baseline_accuracy:.3f} ± {np.std(baseline_scores):.3f}")
    except Exception as e:
        print(f"Ошибка при обучении бейзлайн модели: {str(e)}")
        baseline_scores = np.array([0.5, 0.5, 0.5])
        baseline_accuracy = 0.5

    # Обучение всех моделей
    for model_name, config in models.items():
        try:
            print(f"\n=== ОБУЧЕНИЕ {model_name.upper()} ===")
            
            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('model', config['model'])
            ])
            
            search = RandomizedSearchCV(
                estimator=pipeline,
                param_distributions=config['params'],
                n_iter=5,
                cv=cv,
                scoring='accuracy',
                refit=True,
                n_jobs=1,
                random_state=SEED,
                verbose=1
            )
            
            search.fit(X, y)
            
            cv_scores = cross_val_score(search.best_estimator_, X, y, cv=cv, scoring='accuracy', n_jobs=1)
            mean_accuracy = np.mean(cv_scores)
            std_accuracy = np.std(cv_scores)
            
            results.append({
                'model': model_name,
                'best_params': search.best_params_,
                'mean_accuracy': mean_accuracy,
                'std_accuracy': std_accuracy,
                'best_estimator': search.best_estimator_
            })
            
            print(f"Best params: {search.best_params_}")
            print(f"Accuracy: {mean_accuracy:.3f} ± {std_accuracy:.3f}")
            
        except Exception as e:
            print(f"Ошибка при обучении {model_name}: {str(e)}")
            continue
    
    if results:
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values('mean_accuracy', ascending=False)
        results_df.to_csv('models_evaluation.csv', index=False)
        
        print("\n=== РЕЗУЛЬТАТЫ МОДЕЛЕЙ ===")
        print(results_df[['model', 'mean_accuracy', 'std_accuracy']])
        
        return results_df, baseline_scores
    else:
        print("Ни одна модель не была успешно обучена")
        return pd.DataFrame(), baseline_scores

def main():
    print("=== ЗАПУСК ЭКСПЕРИМЕНТА ===")
    
    # Установите необходимые библиотеки если их нет
    try:
        import chardet
    except ImportError:
        print("Установка chardet...")
        os.system("pip install chardet")
        import chardet
    
    try:
        from charset_normalizer import from_bytes
    except ImportError:
        print("Установка charset-normalizer...")
        os.system("pip install charset-normalizer")
        from charset_normalizer import from_bytes
    
    # 1. Загрузка данных
    train, test = safe_load_data()
    if train.empty:
        print("Ошибка: Не удалось загрузить тренировочные данные")
        return
    
    # 2. Предобработка
    X_train, y_train, X_test = preprocess_data(train, test)
    if X_train.empty or y_train.empty:
        print("Ошибка: Проблемы с предобработкой данных")
        return
    
    # 3. Разделение на train/validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, 
        y_train, 
        test_size=0.3, 
        random_state=SEED,
        stratify=y_train
    )
    
    print(f"Данные загружены: Train {X_train.shape}, Val {X_val.shape}")
    
    # 4. Обучение и оценка моделей
    print("\n=== ОБУЧЕНИЕ МОДЕЛЕЙ ===")
    preprocessor = create_preprocessor()
    models = get_models()
    results_df, _ = train_and_evaluate(X_train, y_train, preprocessor, models)
    
    if results_df.empty:
        print("Не удалось обучить модели. Используем простую логистическую регрессию.")
        simple_model = Pipeline([
            ('preprocessor', preprocessor),
            ('model', LogisticRegression(max_iter=1000, random_state=SEED))
        ])
        simple_model.fit(X_train, y_train)
        best_model = simple_model
        best_model_name = "LogisticRegression (fallback)"
    else:
        best_model_info = results_df.iloc[0]
        best_model = best_model_info['best_estimator']
        best_model_name = best_model_info['model']
        print(f"\nЛучшая модель: {best_model_name}")
        print(f"Validation Accuracy: {best_model_info['mean_accuracy']:.4f}")
    
    # 6. Оценка на валидационном наборе
    val_predictions = best_model.predict(X_val)
    val_accuracy = accuracy_score(y_val, val_predictions)
    print(f"\nФинальная Validation Accuracy: {val_accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_val, val_predictions))
    
    # 7. Работа с тестовыми данными
    if not X_test.empty:
        print("\n=== ОБРАБОТКА ТЕСТОВЫХ ДАННЫХ ===")
        test_predictions = best_model.predict(X_test)
        
        submission = pd.DataFrame({
            'id': X_test.index,
            'prediction': test_predictions,
        })
        
        if not test.empty and 'successful' in test.columns:
            public_accuracy = accuracy_score(test['successful'], test_predictions)
            print(f"Test Accuracy: {public_accuracy:.4f}")
    
    # 8. Сохранение модели
    try:
        joblib.dump(best_model, 'best_model.pkl')
        print("\nМодель сохранена как 'best_model.pkl'")
    except Exception as e:
        print(f"Ошибка при сохранении модели: {str(e)}")

    # Создание CSV файла с предсказаниями
    try:
        if not X_test.empty:
            submission_df = pd.DataFrame({
                'id': X_test.index,
                'successful': test_predictions,
            })
        
        
        submission_df.to_csv('submission.csv', index=False, encoding='utf-8-sig', sep=';')
        print("Предсказания сохранены как 'submission.csv'")
    except Exception as e:
        print(f"Ошибка при создании submission.csv: {str(e)}")

    print("\n=== ЭКСПЕРИМЕНТ ЗАВЕРШЕН ===")
    print(f"Лучшая модель: {best_model_name}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")

# Запуск основной функции
if __name__ == "__main__":
    main()
    

=== ЗАПУСК ЭКСПЕРИМЕНТА ===
ЗАГРУЗКА ОБУЧАЮЩИХ ДАННЫХ...

АНАЛИЗ КОДИРОВКИ: ads.csv
1. Chardet:          ascii (уверенность: 100.00%)
Ошибка в методе 2 (charset-normalizer): 'CharsetMatch' object has no attribute 'confidence'
2. Charset-Normalizer: Не определена (уверенность: 0.00%)
3. Перебор кодировок:  utf-8 (уверенность: 100.00%)

РЕКОМЕНДУЕМАЯ КОДИРОВКА: ascii
(определено методом: Chardet, уверенность: 100.00%)
✓ Файл успешно загружен с кодировкой: ascii

ЗАГРУЗКА ТЕСТОВЫХ ДАННЫХ...

АНАЛИЗ КОДИРОВКИ: test.tsv.csv
1. Chardet:          ascii (уверенность: 100.00%)
Ошибка в методе 2 (charset-normalizer): 'CharsetMatch' object has no attribute 'confidence'
2. Charset-Normalizer: Не определена (уверенность: 0.00%)
3. Перебор кодировок:  utf-8 (уверенность: 100.00%)

РЕКОМЕНДУЕМАЯ КОДИРОВКА: ascii
(определено методом: Chardet, уверенность: 100.00%)
✓ Файл успешно загружен с кодировкой: ascii
Данные загружены: Train (564, 4), Val (242, 4)

=== ОБУЧЕНИЕ МОДЕЛЕЙ ===

=== БЕЙЗЛАЙН МОДЕЛЬ =